In [3]:
import geojson
import geopandas
import pandas as pd
import numpy as np
import statistics as st
import matplotlib as plt
import scipy
from scipy import stats
from scipy.stats import describe
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import folium
import csv
from IPython.display import display, HTML
from pathlib import Path 
import os
from datetime import date, time, datetime
from dateutil import tz #for dealing with timezone
from geopy import distance

/Users/stateofplace/.virtualenvs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [9]:
#get today time
today = date.today()
now = datetime.now()
current_time = time(now.hour, now.minute, now.second)
date = datetime.combine(today, current_time)
new_date = str(date)
new_new_date = new_date.replace(" ", "_")
new_new_date = new_new_date.replace("-", "_")
new_new_date = new_new_date.replace(":", "_")
print(new_new_date)

2022_03_19_14_15_34


In [4]:
#load raw data set from inside airbnb downloaded jan 27 2022
dg = pd.read_csv("florence_listings_jan_27.csv")
# print(dg.head())
# print(dg.size)
# print(type(dg))
#print(dg.columns) 


In [5]:
#clean pandas dataframe by dropping rows we don't care about
dg.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)
#see new size
# print(dg.size)

In [6]:
#convert price from string to number
## convert column "a" of a DataFrame
# df["a"] = pd.to_numeric(df["a"])
print(type(dg['price']))
print((dg['price']))
dg['price'] = dg['price'].str.replace("$", "").str.replace(',', '')
dg['price'] = pd.to_numeric(dg['price'])
print(type(dg['price']))


<class 'pandas.core.series.Series'>
0         $70.00
1         $70.00
2        $135.00
3        $120.00
4        $139.00
          ...   
10263    $300.00
10264    $550.00
10265    $193.00
10266    $400.00
10267     $95.00
Name: price, Length: 10268, dtype: object
<class 'pandas.core.series.Series'>


/var/folders/9h/m6366gw17qd8jtbtczp7ttnc0000gp/T/ipykernel_45302/4067971899.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dg['price'] = dg['price'].str.replace("$", "").str.replace(',', '')


In [7]:
dg['price'] = pd.to_numeric(dg['price'])
print(type(dg['price']))
print(dg['price'])

<class 'pandas.core.series.Series'>
0         70.0
1         70.0
2        135.0
3        120.0
4        139.0
         ...  
10263    300.0
10264    550.0
10265    193.0
10266    400.0
10267     95.0
Name: price, Length: 10268, dtype: float64


In [17]:
#check to make sure everything we want is there in new dataframe
# print(dg.columns)
# print(dg.head())

In [10]:
#save to file clean
filepath = Path('Out_CSV/florence_listings_jan_27_cleaned_' + new_new_date + '.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
dg.to_csv(filepath)
print(filepath)

Out_CSV/florence_listings_jan_27_cleaned_2022_03_19_14_15_34.csv


In [11]:
#trimmed data set 
# data = dg.loc[:2500, :]
data = dg.loc[:500, :]
listings_location = dg.loc[:50, :]

In [20]:
# print(data.size)
# print(data)

In [12]:
#save trimmed and cleaned to csv file in folder in thesis 
filepath = Path('Out_CSV/florence_listings_jan_27_cleaned_trimmed500_' + new_new_date + '.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
data.to_csv(filepath)


In [13]:
#save trimmed 50 and cleaned to csv file in folder in thesis 
filepath = Path('Out_CSV/florence_listings_jan_27_cleaned_trimmed50_' + new_new_date + '.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
listings_location.to_csv(filepath)
print(filepath)


Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_03_19_14_15_34.csv


In [14]:
discrete_data = ["host_total_listings_count","accommodates","bedrooms", "availability_30","availability_60","availability_90", "availability_365","number_of_reviews_ltm","number_of_reviews_l30d", "calculated_host_listings_count"]

In [15]:
#figure out variable for total listings 
#print(dg['id'])
index = dg.index
number_of_rows = len(index)
#print(number_of_rows)
#print(type(dg.index))
idx = dg.index
index_series = idx.to_series()
index_list = idx.to_list()
#print(index_list, index_series)

In [16]:
#make another csv file with neighbhorhood dictionary items, by 5 neighborhoods, pivot table by neighborhood i think, all data points 
#trying to get a pivot table where the index is neighborhood, and the values is total number listings, tried, 'id', index... not working 
pivot_discrete_neighbourhood = pd.pivot_table(data=data, index=["neighbourhood_cleansed"],columns=["room_type"], values='host_total_listings_count', aggfunc='mean')
# print(type(pivot_discrete_neighbourhood))
# print(pivot_discrete_neighbourhood)
print(pivot_discrete_neighbourhood.head())

room_type               Entire home/apt  Hotel room  Private room  Shared room
neighbourhood_cleansed                                                        
Campo di Marte                 2.937500         NaN      2.444444          NaN
Centro Storico                19.131661    5.555556      6.079365          NaN
Gavinana Galluzzo              9.538462    4.000000      2.666667          NaN
Isolotto Legnaia               2.000000         NaN      1.500000          NaN
Rifredi                        1.923077         NaN      3.000000          4.0


In [17]:
#send pivot table dataframe to csv
filepath = Path('/Users/stateofplace/new_codes/thesis_app/Out_CSV/pivot_neighborhood_test.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
pivot_discrete_neighbourhood.to_csv(filepath)


In [18]:
#try groupby instead 
#dfsum = df.groupby('State', as_index=False).sum()
dgsum = dg.groupby('neighbourhood_cleansed', as_index=False).sum()
#print(dgsum)
#not really what I want.. .I think I need to loop throguh and create a dictionary like we thought from beginning

In [28]:
#create my neighborhood dictionary - showing by neighbhorhood, which has intensity for commercial and tax evasion
#for row in csvreader, if neighborhood_cleansed = x then x_count +=1
#if is_entire is true then is_entire +=1,
#continue for: is_hotels, revenue summed, instant_bookable, has_license, 
#then nested neighborhood_dict = {x:{total_count:count, total_entire:, total }, y:{total_count:count, },z:{total_count:count, }...}

In [29]:
#create my host id dictionary, show which hosts are most likely to be commerical or biggest neg impact or tax evasion

In [19]:
#try geopy distance
#import geopy.distance
duomo_coords = (43.7731, 11.2560)
listing_coords = (52.406374, 16.9251681)

print (distance.distance(duomo_coords, listing_coords).km)

1047.9437250143862


importing a shapefile into python, then we'll loop through all the listings points to see which polygon its in, we'll loop through all polygons with sjoin or spatial join, then when it's true we'll stop and for the census tract variable for that listing point, we'll assign the variable to the name of the polygon or the sez2011 column. we might have to make a path with the polygon, then do 'polypath.contains_point(point)'

Definitely cut the shapefile to excluse non flroence. so not 30k polygons. 

In [21]:
# #with geojson opens a geojson file
# with open('census_clipped_R09_11_WGS84.geojson') as f:
#     gj = geojson.load(f)
# # features = gj[]

In [23]:
#doesn't work i think the projection issue with csv file 
# #dataframe of airbnb into geo
# #dj is the census geojson file and airbnb_geo is the airbnb csv file
# #with geopandas opens a geojson file 
# dj = geopandas.read_file('census_clipped_R09_11_WGS84.geojson')
# #print(type(dj))
# airbnb_df = pd.read_csv('Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_02_28_11_46_25_new_variables.csv')
# airbnb_geo = geopandas.GeoDataFrame(airbnb_df, geometry=geopandas.points_from_xy(airbnb_df.longitude, airbnb_df.latitude, crs="EPSG:32632"))
# #print(airbnb_geo.head())
# #print(dj.head())
# dj.crs = "EPSG:32632"
# dj.to_crs(32632) 
# airbnb_geo.crs = "EPSG:32632"
# airbnb_geo.to_crs(32632)
# # dj.to_crs(4326)
# listings_census_data = geopandas.sjoin(dj, airbnb_geo)
# listings_census_data.head()

In [ ]:
# #awesome thing... 
# airbnb_geo.explore()
# dj.explore()

In [24]:
air_census_joined = geopandas.read_file('airbnb_census_joined.shp')
# print(air_census_joined['SEZ2011'])
# air_census_joined.head()
air_census_joined.iloc[:5]

,field_1,id,listing_ur,name,host_id,host_name,host_since,host_locat,host_total,neighbourh,...,E23,E24,E25,E26,E27,E28,E29,E30,E31,geometry
0,0,24469,https://www.airbnb.com/rooms/24469,Fortezza/City Centre Modern Apt 2+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,0,1,2,0,33,2,1,0,0,POINT (11.24392 43.78210)
1,1,24470,https://www.airbnb.com/rooms/24470,Fortezza/City Centre Modern Apt 2+1,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
2,2,24471,https://www.airbnb.com/rooms/24471,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
3,3,24472,https://www.airbnb.com/rooms/24472,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
4,4,26738,https://www.airbnb.com/rooms/26738,N4U Guest House Florence,113883,N4U Guest House,2010-04-26,"Florence, Toscana, Italy",2,Centro Storico,...,1,0,1,0,13,0,2,0,0,POINT (11.25754 43.77017)


In [ ]:
#air_census_joined.explore()

In [ ]:
#playing with loc and iloc etc 
# air_census_joined.shape
# air_census_joined.iloc[[2,20,45]]
# air_census_joined.loc[1]
# air_census_joined.iloc[:,[2,20,45]].head()
# air_census_joined.loc[:,['id','SEZ2011','price']].head()
# rows = [2, 20, 45]
# cols = ['id','SEZ2011','price']
# air_census_joined.loc[rows, cols]
# air_census_joined.loc[1,'SEZ2011' ]
# air_census_joined.iloc[2,34]
air_census_joined.iloc[:5]
# air_census_joined.loc[rows, 'SEZ2011']
# air_census_joined.columns.get_loc('SEZ2011')
# air_census_joined.columns.get_loc('id')
# air_census_joined.iloc[:2, 1:46]
# air_census_joined.iloc[:2, [1,46]]
# air_census_joined.iloc[2, 46]
# air_census_joined.at[0, 'price']
# air_census_joined.iat[4, 46]

,field_1,id,listing_ur,name,host_id,host_name,host_since,host_locat,host_total,neighbourh,...,E23,E24,E25,E26,E27,E28,E29,E30,E31,geometry
0,0,24469,https://www.airbnb.com/rooms/24469,Fortezza/City Centre Modern Apt 2+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,0,1,2,0,33,2,1,0,0,POINT (11.24392 43.78210)
1,1,24470,https://www.airbnb.com/rooms/24470,Fortezza/City Centre Modern Apt 2+1,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
2,2,24471,https://www.airbnb.com/rooms/24471,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
3,3,24472,https://www.airbnb.com/rooms/24472,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
4,4,26738,https://www.airbnb.com/rooms/26738,N4U Guest House Florence,113883,N4U Guest House,2010-04-26,"Florence, Toscana, Italy",2,Centro Storico,...,1,0,1,0,13,0,2,0,0,POINT (11.25754 43.77017)


In [25]:
#change so don't close file everytime and don't have to repeat
#make new variables
# file_name = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_' + new_new_date + '.csv'
# file_name_var = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_' + new_new_date + '_new_variables.csv'
#remember you added a column for date... so really old files won't work 
file_name = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_03_18_11_54_15.csv'
file_name_var = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_03_18_11_54_15_new_variables.csv'

all = []
with open(file_name, 'r') as file:
    with open(file_name_var, 'w') as newfile:
        csvwriter = csv.writer(newfile,lineterminator='\n')
        csvreader = csv.reader(file)
        #extract headers
        header = next(csvreader)
        new_headers = ['days_rented', 'rounded_revenue', 'is_hotel', 'is_entire', 'many_listings', 'only_1_listing', 'only_2_listings', 'host_florence', 'has_liscense', 'is_instant_bookable', 'global_total_listings', 'dist_duomo', 'is_centro', 'is_gavinana', 'is_isolotto', 'is_rifredi', 'is_campo', 'census_tract_id', 'listing_revenue_exceed_LTR', 'effected_by_policy_1', 'effected_by_policy_2', 'effected_by_policy_3', 'commercial', 'num_residents_census', 'num_households_census', 'res_buildings_census', 'num_bedrooms_census', 'num_household_renters_census', 'num_household_owners_census']
        for item in new_headers:
            header.append(item)
        all.append(header)

        for row in csvreader:
            #number_listings_outside_florence
            #number_listings_outside_florence = 'host_total_listings_count' - 'calculated_host_listings_count'
            #didn't work, some were negative, not sure but I think it's mostly correct based off of this sample he has over 200 total but only 10ish in florence which is waht the data says https://www.airbnb.com/s/homes?host_id=10990422&map_toggle=true
            # number_listings_outside_florence = int(row[8]) - int(row[24])  
            #index row 
            listing_row_id = int(row[0])
            # total global and local listings
            global_total_listings = int(row[9]) + int(row[25]) 
            #dist_duomo
            duomo_coords = (43.7731, 11.2560)
            #listing_coords = lat, long
            listing_coords = (row[11], row[12])
            ##dist_duomo = duomo_coords - listing_coords
            dist_duomo = (distance.distance(duomo_coords, listing_coords).m)
            #has_liscense
            if len(row[23]) > 0:
                has_liscense = 1
            else:
                has_liscense = 0
            #is_instant_bookable
            if row[24] == 't':
                is_instant_bookable = 1
            else:
                is_instant_bookable = 0
            #getting info for "host_florence" success
            if ("Florence" or "Firenze") in row[8]:
                host_florence = 1
            else:
                host_florence = 0
                #print(host_florence)
            #getting info for "is_centro" success
            #get better average rent data just guessed from idealista
            if "Centro" in row[10]:
                is_centro = 1
                neighborhood_rent = 1000 * 12
            else:
                is_centro = 0
            #is_campo success
            if "Campo" in row[10]:
                is_campo = 1
                neighborhood_rent = 700 * 12
            else:
                is_campo = 0
            #is_rifredi success
            if "Rifredi" in row[10]:
                is_rifredi = 1
                neighborhood_rent = 600 * 12
            else:
                is_rifredi = 0
            if "Isolotto" in row[10]:
                is_isolotto = 1
                neighborhood_rent = 400 * 12
            else:
                is_isolotto = 0
            if "Gavinana" in row[10]:
                is_gavinana = 1
                neighborhood_rent = 900 * 12
            else:
                is_gavinana = 0
            #many_listings variable success
            if int(row[9]) > 2:
                many_listings = 1
            else:
                many_listings = 0
            #only_1 success
            if int(row[9]) == 1:
                only_1_listing = 1
            else: 
                only_1_listing = 0
            #only_2 success
            if int(row[9]) == 2:
                only_2_listings = 1
            else:
                only_2_listings = 0
            #is_entire success 
            if row[13] == 'Entire home/apt':
                is_entire = 1
            else:
                is_entire = 0
            #is_hotel success
            if row[13] == 'Hotel room':
                is_hotel = 1
            else:
                is_hotel = 0
            #days_rented
            days_rented = (int(row[21])/.7)*2.6
            #revenue #have to strip the string for price then turn into float
            price_stripped_dollar = row[16].strip('$')
            price_replaced_comma = price_stripped_dollar.replace(',','')
            new_price = float(price_replaced_comma)
            #yearly revenue
            revenue = days_rented * new_price
            rounded_revenue = round(revenue, 2)

            #census_tract_id_sez2011
            # listing_row_id = int(row[0])
            #print(listing_row_id)
            #print(listing_row_id)
            

            #year and month
            date_scraped = row[3]
            #print(type(date_scraped))
            #print(date_scraped.split('-'))
            year = date_scraped.split('-')[0]
            month = date_scraped.split('-')[1]
            day = date_scraped.split('-')[2]
            #print(day)

            #commercial
            if listings_location.loc[listing_row_id, 'host_total_listings_count'] > 2:
                commercial = 1
            elif host_florence == 0:
                commercial = 1
            else:
                commercial = 0
            #very_likely_commercial
            if listings_location.loc[listing_row_id, 'host_total_listings_count'] > 2 and listings_location.loc[listing_row_id, 'instant_bookable'] == 1:
                very_likely_commercial = 1
            else:
                very_likely_commercial = 0

            #effected_by_policy_x (1-liscense,2-day limit 90, 3-2 listing max)
            if license == 0:
                effected_by_policy_1 = 1
            else: 
                effected_by_policy_1 = 0
            if int(row[20]) > 90:
                effected_by_policy_2 = 1
            else:
                effected_by_policy_2 = 0
            #man_listings is more than 2 
            if many_listings == 1:
                effected_by_policy_3 = 1
            else:
                effected_by_policy_3 = 0

            #listing_revenue_exceed_LTR
            #if yearly revenue exceeds average rent per neighborhood x12
            yearly_revenue_room = rounded_revenue / float(row[15])
            # print('yearly_revenue_room is:' + str(yearly_revenue_room))
            neighborhood_LTR_room = neighborhood_rent
            # print('neighborhood_LTR_room is:' + str(neighborhood_LTR_room))
            if yearly_revenue_room > neighborhood_LTR_room:
                listing_revenue_exceed_LTR = 1
            else:
                listing_revenue_exceed_LTR = 0
            # print('is listing_revenue_exceed_LTR true?:' + str(listing_revenue_exceed_LTR))
            #census_tract_id_sez2011
            census_tract_id = air_census_joined.loc[listing_row_id, 'SEZ2011']
            #print(census_tract_id)
            #census data points 
            columns = ['P1', 'A47', 'A46', 'PF1', 'E3', 'E27', 'SEZ2011']
            #print(air_census_joined.loc[listing_row_id, columns])
            # num_residents_census, num_households_census, res_buildings_census, num_bedrooms_census, num_household_renters_census, num_household_owners_census
            #number of residents per census tract P1
            num_residents_census = (air_census_joined.loc[listing_row_id, 'P1'])
            #number of households per census tract PF1
            num_households_census = (air_census_joined.loc[listing_row_id, 'PF1'])
            #number of residential buildings per census tract E3
            res_buildings_census = (air_census_joined.loc[listing_row_id, 'E3'])
            # number of bedrooms per census tract E27
            num_bedrooms_census = (air_census_joined.loc[listing_row_id, 'E27'])
            #number of household renters per census tract  A46 
            num_household_renters_census = (air_census_joined.loc[listing_row_id, 'A46'])
            #number of household homeowners per census tract A47
            num_household_owners_census = (air_census_joined.loc[listing_row_id, 'A47'])
            


            #adding new variable to row for each row in file
            new_variables = [days_rented, rounded_revenue, is_hotel, is_entire, many_listings, only_1_listing, only_2_listings, host_florence, has_liscense, is_instant_bookable, global_total_listings, dist_duomo, is_centro, is_gavinana, is_isolotto, is_rifredi, is_campo, census_tract_id, listing_revenue_exceed_LTR, effected_by_policy_1, effected_by_policy_2, effected_by_policy_3, commercial, num_residents_census, num_households_census, res_buildings_census, num_bedrooms_census, num_household_renters_census, num_household_owners_census]
            for item in new_variables:
                row.append(item)
            #adding entire row to all 
            all.append(row)
        #writing all rows to new out file
        csvwriter.writerows(all)

In [ ]:
#deduped full list of census tracts 


In [60]:
#TODO loop through all census tracts and create a list... so can see for each tract, how many listings there are

file_name_var = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_03_18_11_54_15_new_variables.csv'
file_name_var_census = 'Out_CSV/florence_listings_jan_27_cleaned_trimmed50_2022_03_18_11_54_15_new_variables_census.csv'
all = []
census_all = []
census_dict = dict() #count how many listings in each census tract 
census_bedrooms_dict = dict() #count how many airbnb bedrooms in each census tract 
census_accomodates_dict = dict() #count how many accomodates in each census tract
#of those STRs are available +90days, by a nonlocal, entire, or multilister or commercial
# census_entire_dict = dict() #count how many airbnb entire listings in each census tract 
# census_dict_nonlocal= dict()#count how many nonlocal listings in each census tract
# census_dict_commercial= dict() #count how many commercial listings in each census tract
# census_dict_90days= dict() #count how many >90daysavail listings in each census tract
# print(census_dict)
with open(file_name_var, 'r') as file:
    with open(file_name_var_census, 'w') as newfile:
        csvwriter = csv.writer(newfile,lineterminator='\n')
        csvreader = csv.reader(file)
        #extract headers
        header = next(csvreader)
        new_headers = ["census tract"]
        for item in new_headers:
            header.append(item)
        all.append(header)
        census_all.append(new_headers)

        for row in csvreader:
            #loop through all rows of listings
            listing_row_id = int(row[0])
            listing_id = int(row[1])
            #print(listing_row_id)
            columns = ['P1', 'A47', 'A46', 'PF1', 'E3', 'E27', 'SEZ2011']
            #print(air_census_joined.loc[listing_row_id, columns])
            census_tract_id = air_census_joined.loc[listing_row_id, 'SEZ2011']
            #if the listing row's census tract is new, create new dict item where the key is the census tract value is a list of the row
            #or try making it a bunch of lists where the name of the list is related to the census tract value
            # census_dict[census_tract_id] = row

            count_listings = 0
            listings_list = []
            bedroom_count = 0
            if census_tract_id not in census_dict:
                # census_dict[census_tract_id] = row
                count_listings = 1
                census_dict[census_tract_id] = 1
                listings_list = [listing_id]
                #print(air_census_joined.loc[listing_row_id,'bedrooms'])
                print('yes')
                bedroom_count = air_census_joined.loc[listing_row_id,'bedrooms']
                print(bedroom_count)
                # census_dict[census_tract_id] = {'listing_id': listing_id, 'count_listings' : count_listings}
                #d2 = {census_tract_id:{'count_listings' : count_listings}}
                # d2 = {'count_listings' : count_listings}
            else:
                # census_dict[census_tract_id].append(row) #doesn't work i'm not sure why just wanted to append each row ... ok
                count_listings += 1
                census_dict[census_tract_id] +=1
                listings_list.append(listing_id)
                print('not')
                #print(air_census_joined.loc[listing_row_id,'bedrooms'])
                #print(type(air_census_joined.loc[listing_row_id,'bedrooms']))
                bedroom_count += air_census_joined.loc[listing_row_id,'bedrooms']
                # d2 = {census_tract_id:{'count_listings' : count_listings}}
                # census_dict[census_tract_id] = {'listing_id': listing_id, 'count_listings' : count_listings}
                # d2 = {'count_listings' : count_listings}
                #print(listings_list)

                print(bedroom_count) #look into how to do a total bedrooom count thing)


            #census_entire_dict = dict()
            # if is_entire == 1:
            #     if census_tract_id not in census_entire_dict:
            #         # census_dict[census_tract_id] = row
            #         census_entire_dict[census_tract_id] = 1
            #     else:
            #         census_entire_dict[census_tract_id] +=1
            # else:
            #     pass
            # #print(is_entire)
            # if host_florence == 1:
            #     if census_tract_id not in census_dict_nonlocal:
            #         # census_dict[census_tract_id] = row
            #         census_dict_nonlocal[census_tract_id] = 1
            #     else:
            #         census_dict_nonlocal[census_tract_id] +=1
            # else:
            #     pass
            # print(host_florence)
            # d2 = {'count_listings' : count_listings}
            #census_dict[census_tract_id].update(d2) #doesn't work because it's trying to update the value of census dict which is a list
            #census_dict.update(d2) #here we update the actual dict ... but i have a feeling it'll overwrite my rows... :( )
            #adding new variable to row for each row in file
            new_variables = [census_tract_id]
            for item in new_variables:
                row.append(item)
            #adding entire row to all 
            all.append(row)
            census_all.append(new_variables)
        #writing all rows to new out file
        # csvwriter.writerows(all)
        csvwriter.writerows(census_all)
#print(census_dict)
# print(type(census_dict))
# print(census_entire_dict)
#print(census_dict_nonlocal)


yes
1.0
yes
1.0
not
2.0
not
2.0
yes
7.0
yes
1.0
yes
1.0
yes
2.0
yes
1.0
yes
1.0
not
1.0
not
2.0
yes
1.0
not
1.0
not
1.0
yes
1.0
yes
1.0
yes
1.0
not
1.0
not
1.0
yes
1.0
not
1.0
yes
1.0
not
1.0
yes
1.0
yes
1.0
yes
1.0
yes
3.0
yes
1.0
yes
2.0
not
1.0
yes
1.0
yes
1.0
not
1.0
yes
3.0
yes
2.0
yes
1.0
not
1.0
yes
1.0
yes
2.0
not
1.0
not
1.0
yes
1.0
yes
1.0
yes
1.0
yes
2.0
yes
2.0
yes
1.0
not
2.0
not
2.0
yes
2.0


some histogram work ... 

In [ ]:
#scattermix data set from multivariate article
#example 
# data = data_of_interest
# pd.plotting.scatter_matrix(data, diagonal="kde")  #alpha=0.2
# plt.pyplot.tight_layout()
# plt.pyplot.show()

In [ ]:
#histograms of data set
#example from data lab
# hist_age = plt.pyplot.hist(df['Eta4_1'], bins=15)
# print(hist_age)
#other example
#sns.histplot(data=df['age'], stat = 'count' or 'frequency')

In [ ]:
#cross tab frequency of data set 
#example from data lab
# crosstab_interesting_features = pd.crosstab([df['gender'], df['age']], df['civil_status'])
# print(crosstab_interesting_features)
#other example
#tab = (pd.crosstab(index=dg['number_of_reviews_ltm'], columns='frequency')) 
# print(tab)
# tab/tab.sum()

In [ ]:
#normalize or standardize my fields?